In [13]:
from refchecker.utils import get_model_batch_response
import os
import json
import random
import re
from tqdm import tqdm

# 设置API Key
# os.environ['OPENAI_API_KEY'] = 

result = []
change = []
# 模板
prompt_template = """ 
I would like you to help me measure the semantic similarity between two pieces of text. 
I will provide you with 3 texts: reference, text1 and text2. 
You will need to compare text1 and text2 to the reference text and decide which one is more similar to the reference text. Give me a label and a reason for your choice. Label "0" means that text1 is more similar to the reference text, while label "1" means that text2 is more similar to the reference text.

Use the following format for your output:

### Label:
[label]
### Reason:
[Reason]

Now please give your assessment of the semantic similarity between the following texts:

### reference:
[reference]

### text1:
[text1]

### text2:
[text2]

"""


check_template = """
I would like you to help me check the correctness of someone's judgment on the semantic similarity between two pieces of text.
I will provide you with 3 texts: reference, text1 and text2, and the label and reason given by someone else. Label "0" means that text1 is more similar to the reference text, while label "1" means that text2 is more similar to the reference text.
Your task is to decide whether the label and reason are correct or not.
Give me your judgment and a reason for your checking. "True" means that the label are correct, while "False" means that the label are incorrect.

Use the following format for your output:

### Judgment:
True / False

### Reason:

Now please check the correctness of the following judgment:

### reference:
[reference]

### text1:
[text1]

### text2:
[text2]

### Somesone's judgment:
[judgment]

"""

# 加载数据
data_path = 'data/raw.json'
with open(data_path, 'r') as f:
    data = json.load(f)

for i, d in tqdm(enumerate(data)):
    if i<359:
        continue    
    reference = d['reference']  
    text1 = d['text1']
    text2 = d['text2']
    
# 0 -> normal
    if random.random() < 0.5:
        prompt1 = prompt_template.replace('[text1]', text1).replace('[text2]', text2).replace('[reference]', reference)
        prompt2 = check_template.replace('[text1]', text1).replace('[text2]', text2).replace('[reference]', reference)
        change.append(0)
# 1 -> inverse
    else:
        prompt1 = prompt_template.replace('[text1]', text2).replace('[text2]', text1).replace('[reference]', reference)
        prompt2 = check_template.replace('[text1]', text2).replace('[text2]', text1).replace('[reference]', reference)
        change.append(1)    

    # 获取第一个文本对的响应
    responses = get_model_batch_response(
        prompts=[prompt1],
        model='bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0',
        temperature=0,
        max_new_tokens=500
    )
    response_label = responses[0]
    label_match = re.search(r"### Label:\n(\d+)", response_label)

    if label_match:
        label_gpt = label_match.group(1)
        result.append(label_gpt)
    else:
        print(i)
        with open('data/claude_check.json', 'w') as f:
            json.dump(result, f, indent=2)
        with open('data/claude_change.json', 'w') as f:
            json.dump(change, f, indent=2)
        assert 0
    prompt2 = prompt2.replace('[judgment]', response_label) 
    # 获取第二个文本对的响应    
    responses_check = get_model_batch_response(
        prompts=[prompt2],
        model='bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0',
        temperature=0,
        max_new_tokens=500
    )
    response_check = responses_check[0] 
    label_match = re.search(r"### Judgment:\n(\d+)", response_check)
    if label_match:
        check_gpt = label_match.group(1)
        if not check_gpt:
            result[-1]="2"  

# 保存结果到json文件
with open('data/claude_check.json', 'w') as f:
    json.dump(result, f, indent=2)
with open('data/claude_change.json', 'w') as f:
    json.dump(change, f, indent=2)

362it [00:39,  7.38it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


383it [06:06,  1.04it/s]


In [1]:
import json

with open('data/gpt_score.json', 'r') as f:
    gpt_data = json.load(f)

with open('data/raw.json', 'r') as f:
    raw_data = json.load(f)

correct = 0
sum = 0
for i, d in enumerate(raw_data):
    sum += 1    
    d['gpt_score1'] = gpt_data[i][0]
    d['gpt_score2'] = gpt_data[i][1]

    

    if d['gpt_score1'] > d['gpt_score2']:
        d['gpt_label'] = 0
    elif d['gpt_score1'] < d['gpt_score2']:
        d['gpt_label'] = 1
    else:
        d['gpt_label'] = 2  # equal
    
    if d['gt_labels'] == d['gpt_label']:
        correct += 1
    
with open('data/gpt_data.json', 'w') as f:
    json.dump(raw_data, f, indent=2)

print(f'Accuracy: {correct/sum}')   

FileNotFoundError: [Errno 2] No such file or directory: 'data/gpt_score.json'

In [14]:
import json

with open('data/claude_check.json', 'r') as f:
    claude_data = json.load(f)
claude_data = [int(x) for x in claude_data] 
with open('data/claude_change.json', 'r') as f:
    change_claude = json.load(f)
   
# for id,i in enumerate(change_claude):
#     if i == 1:
#         claude_data[id] = 1 - claude_data[i]

with open('data/gpt_check.json', 'r') as f:
    gpt_data = json.load(f)
gpt_data = [int(x) for x in gpt_data]
with open('data/gpt_change.json', 'r') as f:
    change_gpt = json.load(f) 
    
for id,i in enumerate(change_gpt):
    if i == 1:
        gpt_data[id] = 1 - gpt_data[i]

def percentage_agreement(a1, a2):
    total = len(a1)
    agree = sum([1 for x, y in zip(a1, a2) if x == y])
    return agree / total

print(percentage_agreement(claude_data, gpt_data))  

0.7916666666666666
